In [1]:
import pandas as pd
df = pd.read_csv('D:\\dados\\bar_data.csv')
df.head(5)


,DateTime,Open,High,Low,Close,Volume,PavioSuperior,PavioInferior,Corpo,Range,...,StochasticoK,StochasticoD,RSI,MACD,MACDSignal,MACDHistogram,Gatilho,atr8,atr14,atr28
0,2023-01-05 05:15:00,12440.00,12447.00,12438.75,12445.75,556,1.25,1.25,5.75,8.25,...,92.660555,56.200360,55.525627,-2.782009,-7.070517,4.288509,0,12.977921,16.162682,20.955504
1,2023-01-05 05:30:00,12446.00,12446.50,12431.75,12435.00,659,0.50,3.25,-11.00,14.75,...,67.145332,57.594962,51.055336,-2.103282,-6.077070,3.973788,0,13.199431,16.061776,20.733879
2,2023-01-05 05:45:00,12435.50,12442.00,12433.25,12437.50,550,4.50,2.25,2.00,8.75,...,50.690650,59.682826,52.022710,-1.348117,-5.131280,3.783163,0,12.643252,15.539506,20.305883
3,2023-01-05 06:00:00,12437.25,12440.50,12434.50,12439.50,411,1.00,2.75,2.25,6.00,...,37.186255,62.024646,52.825989,-0.581556,-4.221335,3.639779,0,11.812846,14.858113,19.794959
4,2023-01-05 06:15:00,12439.75,12449.75,12433.75,12447.75,967,2.00,6.00,8.00,16.00,...,59.375360,65.224063,56.091757,0.683773,-3.240313,3.924086,0,12.336240,14.939676,19.659425


In [2]:
data = df.copy()

# Converter a coluna DateTime para datetime
data['DateTime'] = pd.to_datetime(data['DateTime'])

# Definir a coluna Gatilho com valores iniciais iguais a 0
data['Gatilho'] = 0

# Definir os horários de verão e não verão
horario_verao_inicio = pd.Timestamp('9:40:00').time()
horario_verao_fim = pd.Timestamp('17:00:00').time()

horario_nao_verao_inicio = pd.Timestamp('10:40:00').time()
horario_nao_verao_fim = pd.Timestamp('18:00:00').time()

# Função para determinar se a data está no horário de verão dos EUA
def is_dst(date):
    year = date.year
    # Segundo domingo de março
    dst_start = pd.Timestamp(year, 3, 1) + pd.DateOffset(weeks=2, weekday=6)
    # Primeiro domingo de novembro
    dst_end = pd.Timestamp(year, 11, 1) + pd.DateOffset(weekday=6)
    return dst_start <= date < dst_end

# Aplicar a lógica para definir o valor da coluna Gatilho
for idx, row in data.iterrows():
    current_time = row['DateTime'].time()
    if is_dst(row['DateTime']):
        if horario_verao_inicio <= current_time <= horario_verao_fim:
            data.at[idx, 'Gatilho'] = 1
    else:
        if horario_nao_verao_inicio <= current_time <= horario_nao_verao_fim:
            data.at[idx, 'Gatilho'] = 1

# Contar o número de dias úteis (de segunda a sexta-feira) no dataframe
dias_uteis = data['DateTime'].dt.date[data['DateTime'].dt.weekday < 5].nunique()
print(f"Número de dias úteis no dataframe: {dias_uteis}")




Número de dias úteis no dataframe: 431


In [3]:
# quero saber quantas linhas no dataframe têm Gatilho = 1
num_gatilho = data['Gatilho'].sum()
print(f"Número de linhas com Gatilho = 1: {num_gatilho}")


Número de linhas com Gatilho = 1: 12666


In [4]:
#salva o dataframe em um arquivo csv
data.to_csv('D:\\dados\\bar_M15_V02_data_01-01-2023_a_31-08-2024.csv', index=False)